In [1]:
!pip install transformers
import numpy as np
import pandas as pd
import os
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import transformers
from transformers import RobertaTokenizer, TFRobertaModel
transformers.logging.set_verbosity_error()
import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.8 MB 5.4 MB/s 
     |████████████████████████████████| 182 kB 48.0 MB/s 
     |████████████████████████████████| 7.6 MB 54.4 MB/s 


In [2]:
df_train = pd.read_csv("https://raw.githubusercontent.com/thomascourtney/nlp_final_project/main/data/Poem_classification%20-%20train_data.csv")
df_test = pd.read_csv("https://raw.githubusercontent.com/thomascourtney/nlp_final_project/main/data/Poem_classification%20-%20test_data.csv")

In [3]:
df_train = df_train.dropna(axis=0)
df_train.isnull().sum()

Genre    0
Poem     0
dtype: int64

In [4]:
df_test.isnull().sum()

Genre    0
Poem     0
dtype: int64

In [5]:
display(df_train)
display(df_test)

,Genre,Poem
1,Music,In the thick brushthey spend the...
2,Music,Storms are generous. ...
3,Music,—After Ana Mendieta Did you carry around the ...
4,Music,for Aja Sherrard at 20The portent may itself ...
5,Music,"for Bob Marley, Bavaria, November 1980 Here i..."
...,...,...
836,Environment,Why make so much of fragmentary blue In here a...
837,Environment,"Woman, I wish I didn't know your name. What co..."
838,Environment,"Yonder to the kiosk, beside the creek, Paddle ..."
839,Environment,You come to fetch me from my work to-night Whe...


,Genre,Poem
0,Music,A woman walks by the bench I’m sitting onwith ...
1,Music,"Because I am a boy, the untouchability of beau..."
2,Music,"Because today we did not leave this world,We n..."
3,Music,"Big Bend has been here, been here. Shouldn’t i..."
4,Music,"I put shells there, along the lip of the road...."
...,...,...
145,Environment,"To pick a tulip from the garden, the red one. ..."
146,Environment,We are as clouds that veil the midnight moon; ...
147,Environment,"When pulled, the spider web took another form...."
148,Environment,Whose woods these are I think I know. His hous...


In [6]:
df_train.Genre.value_counts()


Music          238
Death          231
Environment    227
Affection      141
Name: Genre, dtype: int64

In [7]:
df_test.Genre.value_counts()

Affection      100
Environment     25
Death           13
Music           12
Name: Genre, dtype: int64

In [9]:
X_train = df_train['Poem']
X_train

1                    In the thick brushthey spend the...
2         Storms are generous.                       ...
3       —After Ana Mendieta Did you carry around the ...
4       for Aja Sherrard at 20The portent may itself ...
5       for Bob Marley, Bavaria, November 1980 Here i...
                             ...                        
836    Why make so much of fragmentary blue In here a...
837    Woman, I wish I didn't know your name. What co...
838    Yonder to the kiosk, beside the creek, Paddle ...
839    You come to fetch me from my work to-night Whe...
840    You see them through water and glass, (both li...
Name: Poem, Length: 837, dtype: object

In [11]:
y_train = df_train['Genre']
y_train

1            Music
2            Music
3            Music
4            Music
5            Music
          ...     
836    Environment
837    Environment
838    Environment
839    Environment
840    Environment
Name: Genre, Length: 837, dtype: object

In [13]:
X_test = df_test["Poem"]
X_test

0      A woman walks by the bench I’m sitting onwith ...
1      Because I am a boy, the untouchability of beau...
2      Because today we did not leave this world,We n...
3      Big Bend has been here, been here. Shouldn’t i...
4      I put shells there, along the lip of the road....
                             ...                        
145    To pick a tulip from the garden, the red one. ...
146    We are as clouds that veil the midnight moon; ...
147    When pulled, the spider web took another form....
148    Whose woods these are I think I know. His hous...
149    you can make the maples blazejust by stopping ...
Name: Poem, Length: 150, dtype: object

In [14]:
y_test = df_test['Genre']
y_test

0            Music
1            Music
2            Music
3            Music
4            Music
          ...     
145    Environment
146    Environment
147    Environment
148    Environment
149    Environment
Name: Genre, Length: 150, dtype: object

In [15]:
from sklearn.feature_extraction.text import CountVectorizer

count_vector = CountVectorizer(stop_words='english', max_df=100) 

count_vector.fit(X_train)
X_train_count_vector = count_vector.transform(X_train)
X_test_count_vector = count_vector.transform(X_test)

X_train_count_vector.shape, X_test_count_vector.shape

((837, 8169), (150, 8169))

In [17]:
X_train_count_vector, X_test_count_vector

(<837x8169 sparse matrix of type '<class 'numpy.int64'>'
 	with 18814 stored elements in Compressed Sparse Row format>,
 <150x8169 sparse matrix of type '<class 'numpy.int64'>'
 	with 2522 stored elements in Compressed Sparse Row format>)

In [18]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

lr_clf = LogisticRegression(max_iter=5000)
lr_clf.fit(X_train_count_vector, y_train)
pred = lr_clf.predict(X_test_count_vector)

accuracy_score(y_test, pred)

0.34

In [20]:
from sklearn.metrics import confusion_matrix, classification_report, roc_auc_score

print(confusion_matrix(y_test, pred))


[[25 23 21 31]
 [ 2  3  5  3]
 [ 1  6 14  4]
 [ 0  0  3  9]]
              precision    recall  f1-score   support

   Affection       0.89      0.25      0.39       100
       Death       0.09      0.23      0.13        13
 Environment       0.33      0.56      0.41        25
       Music       0.19      0.75      0.31        12

    accuracy                           0.34       150
   macro avg       0.38      0.45      0.31       150
weighted avg       0.67      0.34      0.37       150



(None, None)

In [21]:
print(classification_report(y_test, pred))

              precision    recall  f1-score   support

   Affection       0.89      0.25      0.39       100
       Death       0.09      0.23      0.13        13
 Environment       0.33      0.56      0.41        25
       Music       0.19      0.75      0.31        12

    accuracy                           0.34       150
   macro avg       0.38      0.45      0.31       150
weighted avg       0.67      0.34      0.37       150



In [22]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vector = TfidfVectorizer(stop_words='english', max_df=100)

tfidf_vector.fit(X_train)

X_train_tfidf_vector = tfidf_vector.transform(X_train)
X_test_tfidf_vector = tfidf_vector.transform(X_test)

X_train_tfidf_vector.shape, X_test_tfidf_vector.shape

((837, 8169), (150, 8169))

In [23]:
lr_clf_2 = LogisticRegression(max_iter=5000)
lr_clf_2.fit(X_train_tfidf_vector, y_train)
pred = lr_clf_2.predict(X_test_tfidf_vector)

accuracy_score(y_test, pred)

0.28

In [24]:
from sklearn.pipeline import Pipeline

pipeline = Pipeline([('count_vector', CountVectorizer(stop_words='english', max_df=100)),
                    ('lr_clf', LogisticRegression())])

In [25]:
pipeline.fit(X_train, y_train)
pred = pipeline.predict(X_test)

accuracy_score(y_test, pred)

0.34

In [26]:
from sklearn.svm import SVC

linearsvc = SVC()

pipeline = Pipeline([('count_vector', CountVectorizer(stop_words='english', max_df=100)),
                    ('linearsvc', SVC(kernel='linear'))])

pipeline.fit(X_train, y_train)
pred = pipeline.predict(X_test)

accuracy_score(y_test, pred)

0.3333333333333333

In [27]:
from sklearn.naive_bayes import MultinomialNB

mnb = MultinomialNB()

pipeline = Pipeline([('count_vector', CountVectorizer(stop_words='english', max_df=100)),
                    ('mnb', MultinomialNB())])

pipeline.fit(X_train, y_train)
pred = pipeline.predict(X_test)

accuracy_score(y_test, pred)

0.2866666666666667

In [28]:
from sklearn.neural_network import MLPClassifier

mlp = MLPClassifier(hidden_layer_sizes=(10,10,10), max_iter=5000, random_state=2211)

pipeline = Pipeline([('count_vector', CountVectorizer(stop_words='english', max_df=100)),
                    ('mlp', MLPClassifier(hidden_layer_sizes=(10,10,10), max_iter=5000, random_state=2211))])

pipeline.fit(X_train, y_train)
pred = pipeline.predict(X_test)

accuracy_score(y_test, pred)

0.31333333333333335